In [1]:
import re
import pandas as pd
from collections import Counter
import json
import urllib
import requests
from tqdm import tqdm
import geocoder

In [2]:
xl = pd.ExcelFile('постаматы-ВСП-ПВЗ на 01.12.2022.xlsx')
sheet_names = xl.sheet_names

In [3]:
df = xl.parse(sheet_names[0], skiprows=1)

In [4]:
df.columns = ['Номер постамата','Дата ввода в эксплуатацию',
              'Адрес постамата','Режим работы', 'Аренда', 'Пусто']
df = df.drop(columns=['Пусто'])
df = df.fillna('НЕТ')

In [5]:
def clean_adress(address):
    address = re.sub('^\d+,\s+','',address)
    replace_dict = {
        ' г ':'город ',' д ':' дом ',
        ' ул ':'улица ',' пл ':'площадь ',
        ' ДОМ':'дом ','пр-кт':'проспект ',
        'б-р':'бульвар ','р-н':'район ',
        'обл,':'область,','ст-ца':' станица ',
        ' стр ':' строение ',' им ':'имени ',
        ' к ':' корпус ',' им. ':'имени',
        ' г. ':'город ',' ул. ':' улица ',
        'Респ ':'Республика ',' рп ':'рабочий поселок '
    }
    for key in replace_dict.keys():
        address = address.replace(key,replace_dict[key])
    return re.sub('\s{2,}',' ',address)

In [6]:
def geo(address):
    rel, lat,lon = 0,0,0
    try:
        response = geocoder.arcgis(address).geojson
        rel = response['features'][0]['properties']['score'] #score
        lat = response['features'][0]['properties']['lat']
        lon = response['features'][0]['properties']['lng']
    except Exception as error:
        print(f'{address}:{error}')
    finally:
        return rel, lat, lon

In [7]:
df['Адрес постамата 2'] = df['Адрес постамата'].apply(lambda x: clean_adress(x))
addresses = df['Адрес постамата 2'].tolist()

In [10]:
res = []
for address in tqdm(addresses[3000:6000]):
    rel, lat,lng = geo(address)
    res.append([rel,lat,lng,address])
res_df = pd.DataFrame(res,columns = ['Точность','Широта','Долгота','Адрес'])
res_df.to_excel('Part3.xlsx')

100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [41:59<00:00,  1.19it/s]


In [ ]:
# g = geocoder.arcgis('Ярославская область, Некоузский район , село Новый Некоуз,улица Советская,дом 50')
# g.geojson